#Importing & Installing Necessary Packages

In [ ]:
!pip install ts2vg
!pip install igraph
!pip install pycairo
!pip install cairocffi
!pip install ordpy

In [ ]:
import seaborn as sns
from ts2vg import NaturalVG
from ts2vg import HorizontalVG
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
import ordpy as op

--------

# -------------- **TASK 1** --------------

# Defining Functions for HVG and NVG

In [ ]:
def HVG(dataframe1,axis, subject, activity,Body_Part):
  method = 'HVG'
  plane = 'attr_'+str(axis)
  g = HorizontalVG()
  g.build(dataframe1[plane][1000:2048])
  ig_g = g.as_igraph()
  avg_deg = np.mean(ig_g.degree())
  net_dia = ig_g.diameter()
  avg_len_path = ig_g.average_path_length()
  dict1 = {'Method': method, 'Subject':subject, 'Accelerometer_Axis': axis, 'Average_Degree': avg_deg, 'Network_Diameter': net_dia,'Average_Path_Length': avg_len_path,'Activity': activity, 'Body_Part': Body_Part}
  return dict1  #returning a dictionary which will then be appended to a dataframe

In [ ]:
def NVG(dataframe1, axis, subject, activity,Body_Part):
  method = 'NVG'
  plane = 'attr_'+str(axis)
  g = NaturalVG()
  g.build(dataframe1[plane][1000:2048])
  ig_g = g.as_igraph()
  avg_deg = np.mean(ig_g.degree())
  net_dia = ig_g.diameter()
  avg_len_path = ig_g.average_path_length()
  dict1 = {'Method': method, 'Subject':subject, 'Accelerometer_Axis': axis, 'Average_Degree': avg_deg, 'Network_Diameter': net_dia,'Average_Path_Length': avg_len_path,'Activity': activity, 'Body_Part': Body_Part}
  return dict1 #returning a dictionary which will then be appended to a dataframe

In [ ]:
body_part = []

In [ ]:
def driver(subject, body_part):
  #df = df[0:0]
  columns1 = ['Method', 'Subject', 'Accelerometer_Axis', 'Average_Degree', 'Network_Diameter','Average_Path_Length','Activity', 'Body_Part']
  df = pd.DataFrame(columns = columns1)
  subject = str(subject)
  activity =[] #blank activity list. All applicable activities will be appended below.

  try:
    walking = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_walking_"+body_part+".csv")
    activity.append("Walking")
  except: 
    pass
  
  try:
    running = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_running_"+body_part+".csv")
    activity.append("Running")
  except: 
    pass

  try:
    climb_up = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_climbingup_"+body_part+".csv")
    activity.append("Climbing Up")
  except: 
    pass

  try:
    climb_down = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_climbingdown_"+body_part+".csv")
    activity.append("Climbing Down")
  except: 
    pass

  #print(body_part)
  axis = ['x','y','z']
  for i in axis:
    for j in activity:
      if j == 'Walking':
        df_1 = HVG(walking,i,subject, j, body_part)
        df_2 = NVG(walking,i,subject,j, body_part)

        df = df.append(df_1,ignore_index=True)
        df = df.append(df_2,ignore_index=True)
      elif j == 'Running':
        df_1 = HVG(running,i, subject, j, body_part)
        df_2 = NVG(running,i,subject,j, body_part)

        df = df.append(df_1,ignore_index=True)
        df = df.append(df_2,ignore_index=True)
      elif j == 'Climbing Up':
        df_1 = HVG(climb_up,i, subject, j, body_part)
        df_2 = NVG(climb_up,i,subject,j, body_part)

        df = df.append(df_1,ignore_index=True)
        df = df.append(df_2,ignore_index=True)
      elif j == 'Climbing Down':
        df_1 = HVG(climb_down,i,subject, j, body_part)
        df_2 = NVG(climb_down,i,subject,j, body_part)

        df = df.append(df_1,ignore_index=True)
        df = df.append(df_2,ignore_index=True)
      else: 
        pass
  
  return df


# Driver Code

In [ ]:
columns1 = ['Method', 'Subject', 'Accelerometer_Axis', 'Average_Degree', 'Network_Diameter','Average_Path_Length','Activity']
body_part = ['chest','head']
df_final = pd.DataFrame(columns = columns1)

for i in range(1,16):
  for j in body_part:
    df = driver(i,j)
    df_final = df_final.append(df)



In [ ]:
df_final.head()

,Method,Subject,Accelerometer_Axis,Average_Degree,Network_Diameter,Average_Path_Length,Activity,Body_Part
0,HVG,1,x,3.961832,27,11.722253,Walking,chest
1,NVG,1,x,11.572519,9,4.462960,Walking,chest
2,HVG,1,x,3.956107,22,7.607361,Running,chest
3,NVG,1,x,8.412214,9,4.330417,Running,chest
4,HVG,1,x,3.967557,35,15.459003,Climbing Up,chest


#Scatter Plot 1 - Average Degree Vs. Network Diameter - Walking & Running


In [ ]:
def viz_walk_run(df_tempo, subject, method, axis, body_part):

  df_tempo = df_tempo[df_tempo.Accelerometer_Axis.isin(axis)]
  df_tempo = df_tempo[df_tempo.Activity.isin(['Walking', 'Running'])]
  df_tempo = df_tempo[df_tempo.Subject.isin([subject])]
  df_tempo = df_tempo[df_tempo.Body_Part.isin([body_part])]

  
  
  if method == 'HVG' or method == 'NVG':
    df_tempo = df_tempo[df_tempo.Method.isin([method])]
  elif method == "both":
    pass
  else:
    pass
  
  
  fig = px.scatter(df_tempo, x='Average_Degree', y='Network_Diameter', color='Activity',hover_data=['Subject','Method','Accelerometer_Axis','Average_Path_Length','Activity'], title = "Average Degree Vs. Network Diameter | "  + str(method)+" | Subject "+ str(subject)+" | "+str(body_part))

  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.show()

## Subject 1 - Chest

### Method: NVG
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Average Degree and Network Diameter using accelerometer (mounted on chest) data of the subject. The method used here is, **Natural Visibilty Graph**. The sensor captures data from all 3 axes (x,y and z) which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Method: Natural Visibilty Graph (NVG)
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Walking & Running




In [ ]:
viz_walk_run(df_final, subject = '1', method = 'NVG', axis =['x','y','z'], body_part = "chest")


### Method: HVG
Although, when we switch the Visibility Graph method to Horizontal Visibility Graph (HVG), the results are as follows;
*   Subject: 1
*   Method: Horizontal Visibilty Graph (HVG)
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Walking & Running

In [ ]:
viz_walk_run(df_final, subject = '1', method = 'HVG', axis =['x','y','z'], body_part = "chest")

## Subject 1 - Head

### Method: NVG
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Average Degree and Network Diameter using accelerometer (mounted on chest) data of the subject. The method used here is, **Natural Visibilty Graph**. The sensor captures data from all 3 axes (x,y and z) which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Method: Natural Visibilty Graph (NVG)
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Walking & Running




In [ ]:
viz_walk_run(df_final, subject = '1', method = 'NVG', axis =['x','y','z'], body_part = "head")

### Method: HVG
Although, when we switch the Visibility Graph method to Horizontal Visibility Graph (HVG), the results are as follows;
*   Subject: 1
*   Method: Horizontal Visibilty Graph (HVG)
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Walking & Running

In [ ]:
viz_walk_run(df_final, subject = '1', method = 'HVG', axis =['x','y','z'], body_part = "head")

## Rest of the Subjects

### Method: HVG

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    viz_walk_run(df_final, subject = i, method = 'HVG', axis =['x','y','z'], body_part = k)





### Method: NVG

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    viz_walk_run(df_final, subject = i, method = 'NVG', axis =['x','y','z'], body_part = k)

#Scatter Plot 2 - Average Degree Vs. Network Diameter - Climb Up & Climb Down

In [ ]:
def viz_climb(df_tempo, subject, method, axis, body_part):
  df_tempo = df_tempo[df_tempo.Activity.isin(['Climbing Up', 'Climbing Down'])]
  df_tempo = df_tempo[df_tempo.Accelerometer_Axis.isin(axis)]
  df_tempo = df_tempo[df_tempo.Subject.isin([subject])]
  df_tempo = df_tempo[df_tempo.Body_Part.isin([body_part])]
  
  if method == 'HVG' or method == 'NVG':
    df_tempo = df_tempo[df_tempo.Method.isin([method])]
  elif method == "both":
    pass
  else:
    pass
  
  
  fig = px.scatter(df_tempo, x='Average_Degree', y='Network_Diameter', color='Activity',hover_data=['Subject','Method','Accelerometer_Axis'], title = "Average Degree Vs. Network Diameter | "  + str(method)+" | Subject "+ str(subject)+" | "+str(body_part))

  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.show()

## Subject 1 - Chest

### Method: NVG
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Average Degree and Network Diameter using accelerometer (mounted on chest) data of the subject. The method used here is, **Natural Visibilty Graph**. The sensor captures data from all 3 axes (x,y and z) which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Method: Natural Visibilty Graph (NVG)
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
viz_climb(df_final, subject = '1', method = 'NVG', axis =['x','y','z'], body_part = "chest")

### Method: HVG
Although, when we switch the Visibility Graph method to Horizontal Visibility Graph (HVG), the results are as follows;
*   Subject: 1
*   Method: Horizontal Visibilty Graph (HVG)
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
viz_climb(df_final, subject = '1', method = 'HVG', axis =['x','y','z'], body_part = "chest")

## Subject 1 - Head

### Method: NVG
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Average Degree and Network Diameter using accelerometer (mounted on chest) data of the subject. The method used here is, **Natural Visibilty Graph**. The sensor captures data from all 3 axes (x,y and z) which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Method: Natural Visibilty Graph (NVG)
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
viz_climb(df_final, subject = '1', method = 'NVG', axis =['x','y','z'], body_part = "head")

### Method: HVG
Although, when we switch the Visibility Graph method to Horizontal Visibility Graph (HVG), the results are as follows;
*   Subject: 1
*   Method: Horizontal Visibilty Graph (HVG)
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
viz_climb(df_final, subject = '1', method = 'HVG', axis =['x','y','z'], body_part = "head")

## Rest of the Subjects

### Method: HVG

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    viz_climb(df_final, subject = i, method = 'HVG', axis =['x','y','z'], body_part = k)


### Method: NVG

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    viz_climb(df_final, subject = i, method = 'NVG', axis =['x','y','z'], body_part = k)

----

# -------------- **TASK 2** --------------

In [ ]:
def comp_entropy(df, axis, activity, subject, dimension, delay, body_part, length):
  plane = 'attr_'+str(axis)
  #print(axis,activity,subject,dimension,delay,body_part,length)
  output = op.complexity_entropy(df[plane], dx = dimension, taux = delay)
  entropy = output[0]
  complexity = output[1]
  dict1 = {'Subject': subject, 'Accelerometer_Axis': axis, 'Signal_Length': length, 'Dimension': dimension,'Delay': delay,'Permutation_Entropy': entropy, 'Complexity': complexity, 'Activity': activity, 'Body_Part': body_part}
  return dict1



In [ ]:
def driver2(subject, body_part, length, dimension, delay):
  #df = df[0:0]
  columns1 = [ 'Subject', 'Accelerometer_Axis', 'Signal_Length', 'Dimension','Delay','Permutation_Entropy', 'Complexity', 'Activity', 'Body_Part']
  df = pd.DataFrame(columns = columns1)
  subject = str(subject)
  activity =[]

  try:
    walking = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_walking_"+body_part+".csv")
    activity.append("Walking")
  except: 
    pass
  
  try:
    running = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_running_"+body_part+".csv")
    activity.append("Running")
  except: 
    pass

  try:
    climb_up = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_climbingup_"+body_part+".csv")
    activity.append("Climbing Up")
  except: 
    pass

  try:
    climb_down = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Data/realworld2016_dataset/proband"+subject+"/data/acc_climbingdown_"+body_part+".csv")
    activity.append("Climbing Down")
  except: 
    pass

  #print(body_part)
  axis = ['x','y','z']
  for i in axis:
    for j in activity:
      if j == 'Walking':
        df_1 = comp_entropy(df = walking[:length], axis = i, activity = j, subject = subject, dimension = dimension, delay = delay, body_part = body_part, length = length)
        df = df.append(df_1,ignore_index=True)

      elif j == 'Running':
        df_1 = comp_entropy(df = running[:length], axis = i, activity = j, subject = subject, dimension = dimension, delay = delay, body_part = body_part, length = length)
        df = df.append(df_1,ignore_index=True)

      elif j == 'Climbing Up':
        df_1 = comp_entropy(df = climb_up[:length], axis = i, activity = j, subject = subject, dimension = dimension, delay = delay, body_part = body_part, length = length)
        df = df.append(df_1,ignore_index=True)

      elif j == 'Climbing Down':
        df_1 = comp_entropy(df = climb_down[:length], axis = i, activity = j, subject = subject, dimension = dimension, delay = delay, body_part = body_part, length = length)
        df = df.append(df_1,ignore_index=True)

      else: 
        pass
  
  return df


# Driver Code

In [ ]:
columns2 = [ 'Subject', 'Accelerometer_Axis', 'Signal_Length', 'Dimension','Delay','Permutation_Entropy', 'Complexity', 'Acitivty']
body_part = ['chest','head']
length_list = [1024, 2048, 4096]
dimension_list = [3, 4, 5, 6]
delay_list = [1,2,3]
df_final2 = pd.DataFrame(columns = columns2)

for i in range(1,16):
  for j in body_part:
    for k in length_list:
      for l in dimension_list:
        for m in delay_list:
          df_temp = driver2(subject = i, body_part = j, length = k, dimension = l, delay = m)
          df_final2 = df_final2.append(df_temp)

In [ ]:
df_final2.head()

,Subject,Accelerometer_Axis,Signal_Length,Dimension,Delay,Permutation_Entropy,Complexity,Acitivty,Activity,Body_Part
0,1,x,1024,3,1,0.856561,0.118808,NaN,Walking,chest
1,1,x,1024,3,1,0.906719,0.081484,NaN,Running,chest
2,1,x,1024,3,1,0.894331,0.091254,NaN,Climbing Up,chest
3,1,x,1024,3,1,0.865585,0.112619,NaN,Climbing Down,chest
4,1,y,1024,3,1,0.814136,0.146812,NaN,Walking,chest


# Scatter Plot 1 - Permutation Entropy Vs. Complexity - Walking & Running

In [ ]:
def ent_walk_run(df_tempo, subject, body_part):

  #df_tempo = df_tempo[df_tempo.Accelerometer_Axis.isin(axis)]
  df_tempo = df_tempo[df_tempo.Activity.isin(['Walking', 'Running'])]
  df_tempo = df_tempo[df_tempo.Subject.isin([subject])]
  df_tempo = df_tempo[df_tempo.Body_Part.isin([body_part])]

  
  fig = px.scatter(df_tempo, x='Permutation_Entropy', y='Complexity', color='Activity',hover_data=['Subject','Accelerometer_Axis','Activity'], title = "Permutation Entropy Vs. Complexity | " + "All Axes" +" | Subject "+ str(subject)+" | "+str(body_part))

  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.show()

### Subject 1: Chest
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Permutation Entropy and Complexity using accelerometer (mounted on chest and head) data of the subject. All 3 axes (x,y and z)  are visualized, which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Walking & Running

In [ ]:
ent_walk_run(df_tempo = df_final2,subject = '1', body_part = 'chest')

### Subject 1: Head
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Permutation Entropy and Complexity using accelerometer (mounted on chest and head) data of the subject. All 3 axes (x,y and z)  are visualized, which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Walking & Running

In [ ]:
ent_walk_run(df_tempo = df_final2,subject = '1', body_part = 'head')

# Scatter Plot 2 - Permutation Entropy Vs. Complexity - Climb Up & Climb Down

In [ ]:
def ent_climb(df_tempo, subject, body_part):

  #df_tempo = df_tempo[df_tempo.Accelerometer_Axis.isin(axis)]
  df_tempo = df_tempo[df_tempo.Activity.isin(['Climbing Up', 'Climbing Down'])]
  df_tempo = df_tempo[df_tempo.Subject.isin([subject])]
  df_tempo = df_tempo[df_tempo.Body_Part.isin([body_part])]

  
  fig = px.scatter(df_tempo, x='Permutation_Entropy', y='Complexity', color='Activity',hover_data=['Subject','Accelerometer_Axis','Activity'], title = "Permutation Entropy Vs. Complexity | " + "All Axes" +" | Subject "+ str(subject)+" | "+str(body_part))

  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.show()

### Subject 1: Chest
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Permutation Entropy and Complexity using accelerometer (mounted on chest and head) data of the subject. All 3 axes (x,y and z)  are visualized, which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Body Part: Chest
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
ent_climb(df_tempo = df_final2,subject = '1', body_part = 'chest')

### Subject 1: Head
In below series of diagrams, we can see various scatter plots for Subject 1.

Firstly, the following digram captures the Permutation Entropy and Complexity using accelerometer (mounted on chest and head) data of the subject. All 3 axes (x,y and z)  are visualized, which can be seen by hovering over the markers. Summary Stats below;

*   Subject: 1
*   Body Part: Head
*   Axes: All Axes
*   Acitivity: Climb Up & Climb Down

In [ ]:
ent_climb(df_tempo = df_final2,subject = '1', body_part = 'head')

## Rest of the Subjects

### Activity - Walking & Running

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    ent_walk_run(df_tempo = df_final2,subject = i, body_part = k)

### Activity - Climb Up & Climb Down

In [ ]:
parts = ['head', 'chest']
for i in range(2,16):
  i = str(i)
  for k in parts:
    ent_climb(df_tempo = df_final2,subject = i, body_part = k)

# Exporting to HTML

In [ ]:
!jupyter nbconvert --to html '/content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Project_2_updated.ipynb'
#<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Project_2_updated.ipynb to html
[NbConvertApp] Writing 3056974 bytes to /content/drive/MyDrive/Colab Notebooks/FDA/Project 2/Project_2_updated.html
